In [33]:
import urllib3
import certifi
from bs4 import BeautifulSoup
from textblob import TextBlob

In [47]:
import urllib3
http = urllib3.PoolManager(
    cert_reqs='CERT_REQUIRED',
    ca_certs=certifi.where())

stock_name = 'AMZN'

r = http.request('GET', 'https://stocktwits.com/symbol/' + stock_name)
r2 = http.request('Get', 'https://twitter.com/search?q=%24' + stock_name + '%23stocks')

In [48]:
#print(r.data.decode('utf-8'))

In [49]:
soup = BeautifulSoup(r.data, 'html.parser')

In [50]:
comment_string = soup.find_all("div", {"class": "st_VwV73eM st_DmhifDD"})

In [51]:
print_string = soup.find_all("span", {"class": "st_3lrv4Jo st_8u0ePN3 st_2oUi2Vb st_31YdEUQ st_8u0ePN3 st_2mehCkH st_3kXJm4P"})

In [52]:
numArrows = 0
pricePos = 0;
for i in range (0,len(str(print_string[0]))):
    if str(print_string[0])[i] == '>':
        numArrows+=1
    if numArrows > 2:
        pricePos = i
        break
        
        
for i in range(pricePos,len(str(print_string[0]))):
    if str(print_string[0])[i] == '<':
        pos = i
        break

print(str(print_string[0])[pricePos+1:pos])

1758.00


In [53]:
sentPos = soup.get_text().find("sentimentChange")+17
for i in range(sentPos, sentPos+7):
    if str(soup.get_text()[i]) == ',':
        sentNumPos = i
        break
        
print(str(soup.get_text()[sentPos:i]))

0.84


In [54]:
soup = BeautifulSoup(r2.data, 'html.parser')

In [55]:
tweet_text = soup.get_text()

In [56]:
tweets = []

parsePos = 0
endParsePos = 0;

for i in range(0, len(tweet_text)-32):
    if tweet_text[i:i+32] == 'Copy link to Tweet\n\n\nEmbed Tweet':
        parsePos = i+32;
        for j in range(i+32, len(tweet_text)-5):
            if tweet_text[j:j+5] == 'Reply':
                endParsePos = j
                tweets.append(tweet_text[parsePos:endParsePos])
                i = j
                break
del tweets[-1]

In [57]:
for i in range(0,len(tweets)):
    tweets[i] = tweets[i].translate({ord('\n'): None})
    tweets[i] = tweets[i][0:len(tweets[i])-24]

In [58]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('brown')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\deark\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\deark\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [59]:
polarityShift = 0;
negative_keywords = ['bear','loss','losing','low','hit','tumble','fell','fall','hurt','recession','worse','sell']
positive_keywords = ['bull','earn','gain','high','great','diversification','well','buy']

total_sentiment = 0;

for tweet in tweets:
    blob = TextBlob(tweet)
    for sentence in blob.sentences:
        polarityShift = 0
        
        for word in negative_keywords:
            if sentence.find(word.lower())!= -1:
                polarityShift-=.3
                
        for word in negative_keywords:
            if sentence.find(word)!= -1:
                polarityShift-=.3
                
        total_sentiment+=polarityShift;
        
        print(sentence)
        print("Score: " + str(sentence.sentiment.polarity + polarityShift))
        print()
        
    print("-----------------------------")


print('\n\n')
print('Overall sentiment ' + str(total_sentiment))

 3 Month Giveaway ($447 Value!)
Score: 0.0

We're giving away 3 months of our service!To Enter!
Score: 0.0

Follow @EliteOptions2 Retweet and Like this post Tag 2 Friends----#GIVEAWAY #GIVEAWAYS #optionstrading #options #stocks #trading$SPY $AAPL $TSLA $GOOGL $AMZN $ZM $SPX $TDOCpic.twitter.com/Pp3kmsEgf898 repl
Score: -0.6

-----------------------------
$AYTU THIS IS HUGE!!!!
Score: 0.9765625000000001

$AMZN $AYTU Has the ONLY AT-HOME TEST!!!
Score: 0.0

$CODX $HTBX $TNXP $BTC $IBIO $TRIL $NOVN $BKYI $CNDT $ENBL $COCP $PCTL $CLDR $ALT $AIM $AHPI $APT $TLSA $ATHE $CORV $SPEX $OAS $SPOM $TTOO $BIOC $INO $OPK #stockshttps://www.cnbc.com/2020/03/11/amazon-care-gates-foundation-may-deliver-coronavirus-tests-in-seattle.html …
Score: 0.6

-----------------------------
$AMZN $NKE $TSLA $SHOP $BAC $BABA Puts all locked in, dead cat bounces make for nice buying opportunities #stocks pic.twitter.com/dxSwFlyzr90
Score: 0.19999999999999998

-----------------------------
Thinking of #stocks that wi